# Clustering
This notebook handles generating and saving clusters that are used in the CB-FDD file.  
We explore three clustering methods in this notebook: Faiss, K-Means, and Mini-Batch K-Means.

## Loading in dependencies and files

In [ ]:
# installing dependencies from SemDeDup Code
!git clone https://github.com/facebookresearch/SemDeDup.git

Cloning into 'SemDeDup'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 24 (delta 2), reused 24 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 1.35 MiB | 8.12 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

file_path = '/content/drive/My Drive/final_train_df.pkl'

with open(file_path, 'rb') as f:
  train_df = pickle.load(f)

train_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,gender,insurance,anchor_age,race
0,0.125711,-1.803007,1.284251,-1.808832,0.127820,-0.191196,0.609272,0.830579,-0.443789,1.238895,...,1.0,0.0,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE
1,0.189455,-1.818042,1.320364,-2.396976,0.120655,-0.262529,0.738478,0.056799,-0.246806,0.450312,...,1.0,0.0,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE
2,-0.466300,-1.172919,0.114043,-1.868590,0.553166,0.417780,0.734430,0.003140,-0.292678,0.740896,...,1.0,0.0,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE
3,-0.507073,-1.562280,0.377472,-1.541743,0.293578,0.306945,0.647620,-0.074701,0.160979,0.329081,...,1.0,0.0,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE
4,-0.200524,-1.818057,0.725834,-2.209781,0.394925,-0.662887,0.430449,0.138520,0.443141,1.002451,...,1.0,0.0,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207309,0.084385,-1.828303,0.807940,-0.793703,0.613876,-0.215462,0.361803,-0.010056,0.053416,1.002149,...,1.0,0.0,0.0,0.0,0.0,0.0,F,Other,19.0,WHITE
207310,-0.327731,-2.051952,1.055242,-0.676235,0.851157,-0.242834,0.080449,0.220710,-0.300547,1.352934,...,1.0,0.0,0.0,0.0,0.0,0.0,F,Other,19.0,WHITE
207311,0.593066,-1.939155,0.648141,-2.380389,0.197016,0.044840,-0.131458,0.692708,-0.744774,1.471385,...,0.0,0.0,0.0,0.0,0.0,0.0,F,Other,57.0,UNKNOWN
207312,-0.033412,-1.678429,0.760117,-2.032235,-0.698559,-0.458091,0.326971,0.787460,0.682498,-0.608661,...,0.0,0.0,0.0,0.0,0.0,1.0,F,Other,57.0,UNKNOWN


In [ ]:
# Select columns of embedding only (recall each embedding vector was a size of 1376 values)
train_df_embedding_columns = train_df.iloc[:, 0:1376]
print(train_df_embedding_columns.shape)
print(train_df_embedding_columns.columns)

(207314, 1376)
Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'feature_1366', 'feature_1367', 'feature_1368', 'feature_1369',
       'feature_1370', 'feature_1371', 'feature_1372', 'feature_1373',
       'feature_1374', 'feature_1375'],
      dtype='object', length=1376)


In [ ]:
# Merge 1376 columns into 1 vector embedding for each row
all_embeddings = train_df_embedding_columns.values
print(len(all_embeddings))
print(all_embeddings)
print(all_embeddings[0].shape)

207314
[[ 0.12571079 -1.8030074   1.2842511  ... -0.7076767   1.0859656
   0.02559637]
 [ 0.18945463 -1.818042    1.3203645  ... -1.3234656   1.4244304
  -0.26066294]
 [-0.4663002  -1.1729188   0.11404295 ... -1.080736    0.6743026
  -0.33340642]
 ...
 [ 0.5930662  -1.9391551   0.6481414  ... -1.0502896  -0.48815688
  -1.756902  ]
 [-0.03341227 -1.678429    0.76011676 ... -1.6180495   0.64806
  -1.2408257 ]
 [ 0.9761496  -2.2109149   0.61055285 ... -1.4995803   0.6610715
  -1.7800003 ]]
(1376,)


## Clustering Methods

### K-Means using Faiss K-Means
This is the library that SemDeDup and FairDeDup used.

In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 15.3 MB/s eta 0:00:00


In [ ]:
D = all_embeddings.shape[1]  # Get embedding dimension

In [ ]:
print(D)

1376


In [ ]:
import faiss
import numpy as np

# Assume embeddings is a (250000, D) NumPy array (D = embedding dimension)
D = all_embeddings.shape[1]  # Get embedding dimension
# num_clusters = 1000
num_clusters = 500
# num_clusters = 1500

# Initialize Faiss KMeans
kmeans = faiss.Kmeans(D, num_clusters, niter=100, verbose=True)

# Train the KMeans model
kmeans.train(all_embeddings.astype(np.float32))  # Faiss requires float32

# Assign clusters
distances, cluster_assignments = kmeans.index.search(all_embeddings.astype(np.float32), 1)

# Convert to Python list
cluster_assignments = cluster_assignments.flatten()

In [ ]:
print(cluster_assignments.shape)

(207314,)


In [ ]:
print(type(cluster_assignments))

<class 'numpy.ndarray'>


Save clusters to a file

In [ ]:
with open('cluster_assignments_faiss','wb') as f: pickle.dump(cluster_assignments, f)

In [ ]:
# with open('cluster_assignments_faiss','rb') as f: clusters = pickle.load(f)

### K-Means using SciKit Learn

In [ ]:
from sklearn.cluster import KMeans

import numpy as np

# kmeans = KMeans(n_clusters=1000, random_state=0, n_init="auto").fit(all_embeddings)
kmeans = KMeans(n_clusters=500, random_state=0, n_init="auto").fit(all_embeddings)
# kmeans = KMeans(n_clusters=1500, random_state=0, n_init="auto").fit(all_embeddings)

In [ ]:
cluster_assignments = kmeans.labels_

In [ ]:
print(cluster_assignments.shape)
print(cluster_assignments)

(207314,)
[393 393 112 ...  53 174 183]


In [ ]:
# print the unique values and counts of all clusters in cluster_assignments
unique_values, counts = np.unique(cluster_assignments, return_counts=True)
print(unique_values)
print(counts)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [ ]:
with open('cluster_assignments_kmeans_500','wb') as f: pickle.dump(cluster_assignments, f)

In [ ]:
with open('cluster_assignments_kmeans_500','rb') as f: clusters = pickle.load(f)

In [ ]:
print(np.equal(cluster_assignments, clusters).all())

True


In [ ]:
# print max value in cluster_assignments
print(np.max(cluster_assignments))
print(np.min(cluster_assignments))

499
0


### Mini-Batch K-Means

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

kmeans = MiniBatchKMeans(n_clusters=1000,
                         random_state=0,
                         batch_size=10000).fit(all_embeddings)

In [ ]:
print(kmeans.cluster_centers_.shape)

(1000, 1376)


In [ ]:
# save to pickle file
with open('cluster_assignments_minibatch.pkl', 'wb') as f: pickle.dump(kmeans.labels_, f)

In [ ]:
with open('cluster_assignments_minibatch', 'rb') as f: kmeans_test = pickle.load(f)
print(kmeans_test)
print(np.equal(kmeans.labels_, kmeans_test.labels_).all())

MiniBatchKMeans(batch_size=10000, n_clusters=1000, random_state=0)
True
